<a href="https://colab.research.google.com/github/beaterjy/demo-word2vec-gensim/blob/master/demo_word2vec_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import jieba
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models import KeyedVectors

In [0]:
data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/training_data_v2_4.xlsx')
data_z = pd.read_excel('/content/drive/My Drive/Colab Notebooks/multi_Z.xlsx')
print(data.shape)
print(data_z.shape)
data_z['phrase'] = data_z['verb'].str.cat(data_z['noun_without_bracket'], sep=' ')


(1922, 8)
(129672, 6)


In [0]:
data.head()

,门类,门类编号,大类,词组,动词,名词,大类编号,子类编号
0,农、林、牧、渔业,A,农业,谷物种植,种植,谷物,1,11
1,农、林、牧、渔业,A,农业,稻谷种植,种植,稻谷,1,111
2,农、林、牧、渔业,A,农业,小麦种植,种植,小麦,1,112
3,农、林、牧、渔业,A,农业,玉米种植,种植,玉米,1,113
4,农、林、牧、渔业,A,农业,其他谷物种植,种植,其他谷物,1,119


In [0]:
data_z.head()

,Unnamed: 0,level,zch,verb,noun,noun_without_bracket,phrase
0,0,0,440704000009296,销售,食品,食品,销售 食品
1,1,0,440704000009296,销售,日用杂品。,日用杂品,销售 日用杂品
2,2,1,440782000021799,自产自销,不锈钢,不锈钢,自产自销 不锈钢
3,3,1,440782000021799,自产自销,铝,铝,自产自销 铝
4,4,1,440782000021799,自产自销,铁,铁,自产自销 铁


In [0]:
#停用词
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file, encoding='gb18030') as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list] #todo: ?
    return custom_stopwords_list

stop_words_file = '/content/drive/My Drive/Colab Notebooks/stopwordsHIT.txt'
stopwords = get_custom_stopwords(stop_words_file)
stopwords_set = frozenset(stopwords)
stopwords[:10]

['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→', '℃ ', '&']

In [0]:
#分词并处理停用词
def chinese_phrase_cut(text):
  text = text.replace(r'、', '')
  segs = jieba.cut(text)
  final = []
  for seg in segs:
    if seg not in stopwords_set:
      final.append(seg)
  return ' '.join(final)

data['phrase_cutted'] = data['词组'].apply(chinese_phrase_cut)
data_z['phrase_cutted'] = data_z['phrase'].apply(lambda x: chinese_phrase_cut(str(x)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.923 seconds.
Prefix dict has been built succesfully.


In [0]:
data.head(20)

,门类,门类编号,大类,词组,动词,名词,大类编号,子类编号,phrase_cutted
0,农、林、牧、渔业,A,农业,谷物种植,种植,谷物,1,11,谷物 种植
1,农、林、牧、渔业,A,农业,稻谷种植,种植,稻谷,1,111,稻谷 种植
2,农、林、牧、渔业,A,农业,小麦种植,种植,小麦,1,112,小麦 种植
3,农、林、牧、渔业,A,农业,玉米种植,种植,玉米,1,113,玉米 种植
4,农、林、牧、渔业,A,农业,其他谷物种植,种植,其他谷物,1,119,谷物 种植
5,农、林、牧、渔业,A,农业,豆类、油料和薯类种植,种植,豆类,1,12,豆类 油料 薯类 种植
6,农、林、牧、渔业,A,农业,豆类种植,种植,豆类,1,121,豆类 种植
7,农、林、牧、渔业,A,农业,油料种植,种植,油料,1,122,油料 种植
8,农、林、牧、渔业,A,农业,薯类种植,种植,薯类,1,123,薯类 种植
9,农、林、牧、渔业,A,农业,棉、麻、糖、烟草种植,种植,棉、麻、糖、烟草,1,13,棉 麻糖 烟草 种植


In [0]:
#构建X
A = data[['phrase_cutted']]
B = data_z[['phrase_cutted']]
X = pd.concat([A, B], axis=0)
X = X.reset_index(drop=True)
#标签y
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(data['大类'])
data['label'] = y

In [0]:
#保存txt
X.to_csv('/content/drive/My Drive/Colab Notebooks/phrase_cutted.txt', header=False, index=False)

In [0]:
X.head()

,phrase_cutted
0,谷物 种植
1,稻谷 种植
2,小麦 种植
3,玉米 种植
4,谷物 种植


In [0]:
#训练word2vec
dim = 300
def train_word2vec():
  inp = '/content/drive/My Drive/Colab Notebooks/phrase_cutted.txt'
  model = Word2Vec(LineSentence(inp), size=dim, window=1, min_count=5, )
  model.save('/content/drive/My Drive/Colab Notebooks/ph_cutted.model')
  model.wv.save_word2vec_format('/content/drive/My Drive/Colab Notebooks/ph_cutted.vector', binary=False)
train_word2vec()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#测试word2vec
# model = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/ph_cutted.model')
model = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/baike_26g_news_13g_novel_229g.model')
# model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/zh.vec')
testwords = ['种植', '零售', '销售' ]
for x in testwords:
  res = model.most_similar(x)
  print(x)
  print(res)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#合并关键词向量
def merge_vec(lst, dim):
  vec = np.zeros(dim)
  if not lst:
    return vec
  count = 0
  for w in lst:
    try:
      vec += model[w]
      count += 1
    except KeyError:
      continue
  vec /= count
  return vec

X_vec = X['phrase_cutted'].apply(lambda x: merge_vec(x.split(), dim))
X_vec = pd.DataFrame(X_vec.values.tolist())
X_vec = X_vec.replace(np.nan, 0)

In [0]:
#正则化
from sklearn.preprocessing import MinMaxScaler
X_vec = pd.DataFrame(MinMaxScaler().fit_transform(X_vec))

In [0]:
#划分训练集和测试集
from sklearn.model_selection import train_test_split
X_t = X_vec.head(len(A))
Z = X_vec.tail(len(B))
X_train, X_test, y_train, y_test = train_test_split(X_t, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(Z.shape)

In [0]:
#贝叶斯模型
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
#决策树
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=1 )
#随机森林
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500, criterion='gini', random_state=1, )
#SVM.LinearSVC
from sklearn.svm import LinearSVC
svc = LinearSVC()
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
#XGboost
from sklearn.ensemble import GradientBoostingClassifier
xgb = GradientBoostingClassifier()

In [0]:
#训练分数
from sklearn import metrics
m = svc
m = m.fit(X_train, y_train)
y_pred = m.predict(X_test)
#训练集分数
print(m.score(X_train, y_train))
#测试准确率
print(metrics.accuracy_score(y_test, y_pred))

In [0]:
#新数据预测结果
Z_pred = m.predict(Z)
data_z['predit'] = encoder.inverse_transform(Z_pred)
data_z.head(50)
# data_z.to_excel('/content/drive/My Drive/Colab Notebooks/Z_pre_v2.xlsx')